In [1]:
#Step 0 Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Step 1 import data
df_raw = pd.read_csv("../data/hour.csv")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [3]:
df_raw.sample(10,random_state=2025)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
3810,3811,2011-06-12,2,0,6,16,0,0,0,3,0.68,0.6364,0.83,0.2836,115,235,350
15985,15986,2012-11-03,4,1,11,18,0,6,0,2,0.36,0.3333,0.46,0.2836,58,239,297
8444,8445,2011-12-23,1,0,12,12,0,5,1,1,0.40,0.4091,0.54,0.5224,14,156,170
8788,8789,2012-01-07,1,1,1,0,0,6,0,1,0.36,0.3485,0.50,0.1642,2,77,79
1925,1926,2011-03-26,2,0,3,0,0,6,0,2,0.26,0.2424,0.38,0.2836,3,25,28
457,458,2011-01-21,1,0,1,2,0,5,1,3,0.24,0.2424,0.75,0.1642,0,2,2
2430,2431,2011-04-16,2,0,4,4,0,6,0,2,0.36,0.3485,0.87,0.2239,1,4,5
16935,16936,2012-12-13,4,1,12,10,0,4,1,1,0.32,0.3030,0.45,0.2836,28,139,167
851,852,2011-02-07,1,0,2,22,0,1,1,1,0.28,0.3030,0.81,0.0896,3,34,37
5181,5182,2011-08-08,3,0,8,19,0,1,1,1,0.82,0.7424,0.43,0.1045,64,352,416


In [26]:
df_baking = df_raw.copy()
df_baking['season'] = df_baking['season'].map({1:"invierno", 2:"primavera", 3:"verano", 4:"otoño"})
df_baking['season'] = df_baking['season'].astype('category')
df_baking['yr'] = df_baking['yr'].map({0:2011, 1:2012})
df_baking['yr'] = df_baking['yr'].astype('category')
df_baking['mnth'] = df_baking['mnth'].astype('category')
df_baking['weekday'] = df_baking['weekday'].map({0:"D", 1:"L",2:"Ma",3:"Mi",4:"J",5:"V",6:"S"})
df_baking['weekday'] = df_baking['weekday'].astype('category')
df_baking['weathersit'] = df_baking['weathersit'].astype('category')
df_baking['temp'] = df_baking['temp'].map(lambda x: x*47-8)
df_baking['atemp'] = df_baking['atemp'].map(lambda x: x*66-16)
df_baking['hum'] = df_baking['hum'].map(lambda x: x*100)
df_baking['windspeed'] = df_baking['windspeed'].map(lambda x: x*67)
df_baking['holiday'] = df_baking['holiday'].map({0:"No", 1:"Si"})
df_baking['holiday'] = df_baking['holiday'].astype('category')
df_baking['workingday'] = df_baking['workingday'].map({0:"No", 1:"Si"})
df_baking['workingday'] = df_baking['workingday'].astype('category')
df_baking['time'] = df_baking['hr'].apply(lambda x: str(x)+":00:00" if x > 9 else "0"+str(x)+":00:00")
df_baking['dteday'] = df_baking['dteday'].astype("str")
df_baking['datetime'] = df_baking['dteday'] + df_baking["time"]
df_baking['datetime'] = pd.to_datetime(df_baking['datetime'],errors="coerce")
df_baking = df_baking.drop(columns=['instant','yr','mnth','hr','dteday','time'])
df = df_baking.copy()
df.info()

/tmp/ipykernel_37762/2885245461.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_baking['datetime'] = pd.to_datetime(df_baking['datetime'],errors="coerce")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   season      17379 non-null  category      
 1   holiday     17379 non-null  category      
 2   weekday     17379 non-null  category      
 3   workingday  17379 non-null  category      
 4   weathersit  17379 non-null  category      
 5   temp        17379 non-null  float64       
 6   atemp       17379 non-null  float64       
 7   hum         17379 non-null  float64       
 8   windspeed   17379 non-null  float64       
 9   casual      17379 non-null  int64         
 10  registered  17379 non-null  int64         
 11  cnt         17379 non-null  int64         
 12  datetime    0 non-null      datetime64[ns]
dtypes: category(5), datetime64[ns](1), float64(4), int64(3)
memory usage: 1.1 MB


In [16]:
df.sample(10)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,time
126,127,2011-01-06,invierno,2011,1,12,No,J,Si,1,4.22,3.0014,35.0,0.0000,6,78,84,12:00:00
127,128,2011-01-06,invierno,2011,1,13,No,J,Si,1,4.22,1.9982,35.0,7.0015,12,55,67,13:00:00
1668,1669,2011-03-15,invierno,2011,3,2,No,Ma,Si,1,6.10,3.9980,62.0,6.0032,1,2,3,02:00:00
13780,13781,2012-08-02,verano,2012,8,9,No,J,Si,1,27.72,32.0018,66.0,7.0015,31,312,343,09:00:00
2050,2051,2011-03-31,primavera,2011,3,7,No,J,Si,3,3.28,-0.9982,93.0,16.9979,5,87,92,07:00:00
2313,2314,2011-04-11,primavera,2011,4,7,No,L,Si,2,15.50,15.9968,88.0,15.0013,12,164,176,07:00:00
3405,3406,2011-05-26,primavera,2011,5,19,No,J,Si,1,28.66,30.0020,46.0,22.0028,67,311,378,19:00:00
14917,14918,2012-09-18,verano,2012,9,18,No,Ma,Si,2,20.20,20.0030,88.0,11.0014,19,262,281,18:00:00
1820,1821,2011-03-21,primavera,2011,3,14,No,L,Si,2,17.38,18.0032,64.0,15.0013,36,65,101,14:00:00
549,550,2011-01-25,invierno,2011,1,1,No,Ma,Si,2,-0.48,-4.9978,69.0,11.0014,0,5,5,01:00:00
